In [ ]:
import os
import sys
l=os.listdir("C:\\mlworkshop\\Others\\Dog Breed Identification(Kaggle)\\train")

In [ ]:
len(l)

In [ ]:
d=dict()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as ms
%matplotlib inline


In [ ]:
df=pd.read_csv("labels.csv")

In [ ]:
df.head()

In [ ]:
def func(s):
    return s+'.jpg'

In [ ]:
df["id"]=df["id"].apply(func)

In [ ]:
df.head()

In [ ]:
breed=list(df["breed"].unique())
breed

In [ ]:
for i in breed:
    values=list(df[df["breed"]==i]["id"])
    d[i]=values

In [ ]:
d

import glob
import shutil
import os
src_dir = "C:\\mlworkshop\\Others\\Dog Breed Identification(Kaggle)\\train"
for i,j in d.items():
    dest_dir="C:\\mlworkshop\\Others\\Dog Breed Identification(Kaggle)\\training_datasets\\"+i
    os.makedirs(dest_dir)
    for k in j:
        for jpgfile in glob.iglob(os.path.join(src_dir,k)):
            shutil.copy(jpgfile, dest_dir)

import glob
import shutil
import os
src_dir = "C:\\mlworkshop\\Others\\Dog Breed Identification(Kaggle)\\train"
for i,j in d.items():
    dest_dir="C:\\mlworkshop\\Others\\Dog Breed Identification(Kaggle)\\training_sample\\"+i
    os.makedirs(dest_dir)
    l=((len(j)*70)//100)
    for k in j[:l]:
        for jpgfile in glob.iglob(os.path.join(src_dir,k)):
            shutil.copy(jpgfile, dest_dir)
    dest_dir="C:\\mlworkshop\\Others\\Dog Breed Identification(Kaggle)\\testing_sample\\"+i
    os.makedirs(dest_dir)
    for k in j[l:]:
        for jpgfile in glob.iglob(os.path.join(src_dir,k)):
            shutil.copy(jpgfile, dest_dir)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [ ]:
base_model = VGG16(#weights='imagenet',
    weights = None, include_top=False, input_shape=(64,64, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(120, activation='softmax')(x)

# This is the model we will train
classifier = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

classifier.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
#classifier.add(Dense(units = 125, activation = 'relu'))
classifier.add(Dense(units = 120, activation = 'softmax'))
classifier.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('training_sample',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('testing_sample',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
total_set = train_datagen.flow_from_directory('training_datasets',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
import h5py
from PIL import Image
from tensorflow.contrib.keras.python.keras import applications
from tensorflow.contrib.keras.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.contrib.keras.python.keras import optimizers
from tensorflow.contrib.keras.python.keras.models import Sequential
from tensorflow.contrib.keras.python.keras.models import Model
from tensorflow.contrib.keras.python.keras.layers import Dropout, Flatten, Dense

In [ ]:
classifier.fit_generator(training_set,steps_per_epoch = 75,epochs = 50,validation_data = test_set,validation_steps = 2)
#classifier.fit_generator(training_set,steps_per_epoch = 50,epochs = 2,validation_data = test_set,validation_steps = 2)

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#scalar=MinMaxScaler()
#scalar.fit(training_set)
#X=scalar.transform(training_set)

In [ ]:
#classifier.fit(X,test_set)

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
prediction = 'dog'
else:
prediction = 'cat'

In [ ]:
import numpy as np
from keras.preprocessing import image

In [ ]:
from os import listdir
l=listdir("C:\\mlworkshop\\Others\\Dog Breed Identification(Kaggle)\\test")
len(l)

In [ ]:
name_dict=training_set.class_indices

In [ ]:
res_dict=dict()
res_dict['id']=[]
key=list(name_dict.keys())
for j in key:
    res_dict[j]=[]

In [ ]:
for i in l:
    test_image = image.load_img('test//'+i, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict_proba(test_image)
    res_dict['id'].append(i[:-4])
    for j in range(120):
        res_dict[key[j]].append(result[0][j])

In [ ]:
!pip install pickle
import pickle

In [ ]:
with open('output2.pickle', 'wb') as f:
    pickle.dump(res_dict, f) # here we are storing/dumping the result into pickle and it act's as a container...ok naah.

In [ ]:
# now load result with the saved result from pickle........
with open('output2.pickle', 'rb') as f:
    b = pickle.load(f) #we reload from the pickle the stored info and obtain the saved result's...

In [ ]:
b

In [ ]:
#predict= classifier.predict_proba(test_set,verbose=2)
df=pd.DataFrame(res_dict)
new_df=df.drop(["id"],axis=1)
result=pd.concat([df['id'],new_df],axis=1)

In [ ]:
result.head()

In [ ]:
result.to_csv("result.csv",index=False)

In [ ]:
'''def func(val):
    return float(val)'''

In [ ]:
'''for j in key:
    result[j]=result[j].apply(func)'''

In [ ]:
result.head()